In [1]:
!pip install requests
!pip install selenium

In [2]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from urllib.parse import urlparse
import webbrowser
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import config
import json

In [3]:
client_id = config.client_id
client_secret = config.client_secret

In [ ]:
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_did_expire = True
    client_id = None
    client_secret = None
    did_perform_grant_flow_auth = False
    authorize_url = "https://accounts.spotify.com/authorize"
    redirected_url = None
    scope = None
    headers = None
    version = "v1"
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def perform_auth_grant_flow(self):
        client_id = self.client_id
        authorize_url = self.authorize_url
        redirect_uri = "http://localhost:8888/callback"
        authorize_url = self.authorize_url
        scope = self.scope
        if scope is None:
            raise Exception("lack of scope, set permissions scope")
        
        parameters = urlencode({"client_id": client_id, "redirect_uri": redirect_uri, "response_type": "token", 
                                "scope": scope})
        request = requests.get(f"{authorize_url}?{parameters}")
        if request.status_code in range(200,299):
            #webbrowser.open(f"{authorize_url}?{parameters}")
            driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
            driver.get(f"{authorize_url}?{parameters}")
            try:
                WebDriverWait(driver, 60).until(EC.url_contains("access"))
            finally:
                url = driver.current_url
                driver.quit();
                return url
        else:
            raise Exception("failed to request")
    
    def set_redirected_url(self, url):
        self.redirected_url = url
        
    def get_redirected_url(self):
        return self.redirected_url
    
    def parse_redirected_url(self, url):
        if 'error' in url:
            raise Exception("user denied access")
            
        
        spotify.set_redirected_url(url)
        query = urlparse(spotify.get_redirected_url())[5]
        parts = query.split('&')
    
        for i in range(len(parts)):
            parts[i] = parts[i].split('=')[1]
        return parts
    
    def set_access_token_and_expire(self, parts):
        self.access_token = parts[0]
        expires_in_seconds = int(parts[2])
        current_time = datetime.datetime.now()
        expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
        self.token_expires = expires
        self.access_token_did_expire = expires < current_time
    
    def get_access_token(self):
        return self.access_token
    
    def get_expires(self):
        return self.token_expires
    
    ### different scopes must be space separated
    def set_scope(self, scope_string):
        self.scope = scope_string
        
    def get_scope(self):
        return self.scope
    
    def set_headers(self, access_token):
        self.headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
        
    def get_headers(self):
        return self.headers
    
    def get_request(self, endpoint):
        headers = self.headers
        
        if headers is None:
            raise Exception("didn't set headers")
            
        r = requests.get(endpoint, headers=headers)
        
        if r.status_code == 204:
            raise Exception("No content in request")
            
        return r
    
    def get_current_user_playlists(self):
        
        if "playlist-read-private" not in self.scope:
            raise Exception("didn't find playlist-read-private in scope")
        
        version = self.version
        endpoint = f"https://api.spotify.com/{version}/me/playlists?limit=50"
        r = self.get_request(endpoint)
        
        return r.json()
    
    def get_playlists_ids(self, playlists_json):
        ids = []
        for item in playlists_json['items']:
            ids.append(item['id'])
        return ids
    
    def get_playlist_tracks(self, playlist_id):
        version = self.version
        endpoint = f"https://api.spotify.com/{version}/playlists/{playlist_id}/tracks"
        r = self.get_request(endpoint)
        return r.json()
    
    def get_tracks_artists_ids(self, tracks_json):
        ids = []
        for item in tracks_json['items']:
            ids.append(item['track']['artists'][0]['id'])
        return ids
    
    def get_artist(self, artist_id):
        version= self.version
        endpoint = f"https://api.spotify.com/{version}/artists/{artist_id}"
        r = self.get_request(endpoint)
        return r.json()
        

In [27]:
spotify = SpotifyAPI(client_id, client_secret);
spotify.set_scope("playlist-read-private")
url = spotify.perform_auth_grant_flow()

In [28]:
query = spotify.parse_redirected_url(url)
spotify.set_access_token_and_expire(query)
spotify.set_headers(spotify.get_access_token())

In [29]:
playlist_json = spotify.get_current_user_playlists();
playlist_ids = spotify.get_playlists_ids(playlist_json)
print(playlist_ids)

['2UA39JwqwiRSXHtfuGbGpk', '37i9dQZF1EttGlFRCrAVsG', '37i9dQZF1E9P2I7X5YoJ37', '37i9dQZF1EjniAoe8LUcSU', '63VZkiWGeT9aBwmSZtpymg', '37i9dQZF1ELZchLZzCHvOS', '5Q3z2p5TmCEdiGOW45VZnZ', '2cig4VycUu3V1FLotxciXE', '0XJDyY2UMcTnZZwRCaAWV4', '0ZBb59skaWIDwJHPFDV92a', '4WlYbStsVuSo4noLAL3kwX', '37i9dQZF1DWXo9v62EXLlM', '7abUEbx189e8lj57YkhMn3', '37i9dQZF1DWWEJlAGA9gs0', '37i9dQZF1DX2LTcinqsO68', '2FSTEf54NbWRZoXFKmxuIj', '01v0GcDMIML985JQs5QwgZ', '4nsNXKrmW9dhFeONFLaEe5', '5Yxx7cxBOoBmeMaMsjyS8C', '4blr5yafyFPhdyREKHQ4go', '37i9dQZF1DX4JAvHpjipBk', '6v0i58u43Xi5MHLS4AuMCa', '2X8xtgNR285fRPWpiXQEP0', '37i9dQZF1DWZeKCadgRdKQ', '05bouYSYuqwmMePAxw2qK3', '37i9dQZEVXcUW1YfiO3Qsa']


In [30]:
tracks_json = spotify.get_playlist_tracks(playlist_ids[0])
artists_ids = spotify.get_tracks_artists_ids(tracks_json)
print(artists_ids)

['3eM69n1zBUNhr7nsDLm5AH', '0MIG6gMcQTSvFbKvUwK0id', '28MEXbuN5228Ig7tTlyEpV', '0MIG6gMcQTSvFbKvUwK0id', '0MIG6gMcQTSvFbKvUwK0id', '4nPxrGG7k7aEKmNLsfX4cd', '4nPxrGG7k7aEKmNLsfX4cd', '4nPxrGG7k7aEKmNLsfX4cd']
